In [ ]:
# ! pip install -q langchain sentence-transformers faiss-cpu langchainhub
# ! pip install geojson
# ! pip install jq
# ! pip install chromadb
# ! pip install open-clip-torch

In [10]:
import os
import torch
import geojson

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline, hub
from langchain.chains import RetrievalQA

from langchain_community.llms import HuggingFaceHub
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import (
    AutoTokenizer, 
    pipeline,
    AutoModelForCausalLM, 
    BitsAndBytesConfig,
    TrainingArguments
)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.vectorstores import Chroma

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [2]:
! nvidia-smi

Thu Apr 11 15:43:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                        Off| 00000000:5E:00.0 Off |                    0 |
| N/A   68C    P0               32W /  70W|   1335MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
# Alternative input with text instead of json

buildings_text_path = 'Data/spb_buildings_text2.txt'
services_text_path = 'Data/spb_services.txt'

with open(buildings_text_path) as file:
    lines1 = [line.rstrip() for line in file]

with open(services_text_path) as file:
    lines2 = [line.rstrip() for line in file]

In [6]:
print(len(lines1))
print(lines1[10000])

print(len(lines2))
print(lines2[10000])

89163
Здание по адресу Санкт-Петербург, Джона Рида, 10 корпус 1 в муниципальном округе Правобережный в районе Невский - это Жилой дом. Его управляющая организация называется ООО "Жилкомсервис № 1 Невского района". Здание было построено по проекту типа 606-м в 1991.0 году. Площадь здания составляет 1955.8050537109375 квадратных метров, а жилая площадь занимает 15780.900390625 квадратных метров. У здания 10.0 этажей, оно рассчитано на 522.0 человек. Сейчас в здании проживает 667.0 человек. В оснащение здания входит: лифты (всего лифтов 7.0) штук, мусоропровод (всего их 1 штук), центральное отопление (1), холодное водоснабжение (1), горячее водоснабжение (1), электричество (1) и газ (0). В здании проводился ремонт в следующих годах: None. Идентификационный номер здания: 122643, идентификационный номер района: 981, находится ли здание в аварийном состоянии: 0. Координаты здания: [30.452412, 59.921978].
14053
В здании по адресу None в муниципальном округе Смольнинское в районе Центральный н

In [ ]:
# Embeddings for adding data to vector DB

embedding_function = SentenceTransformerEmbeddings(model_name="cointegrated/rubert-tiny2")

In [9]:
# Prep data for vector DB

def get_splits(lines):
    documents = [Document(page_content=text) for text in lines]
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(documents)
    return splits

building_splits = get_splits(lines1)
service_splits = get_splits(lines2)

In [ ]:
# Create to vector DBs for buildings and services

db_buildings = '../database/db-buildings'
db_services = '../database/db-services'

buildings_vectorstore = Chroma.from_documents(
    collection_name='db-buildings',
    documents=building_splits,
    embedding=embedding_function4,
    persist_directory=db_buildings
)

buildings_retriever = buildings_vectorstore.as_retriever()

services_vectorstore = Chroma.from_documents(
    collection_name='db-services',
    documents=service_splits,
    embedding=embedding_function4,
    persist_directory=db_services
)

In [ ]:
# Set up LLM

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и отвечаешь на их вопросы. Используй приведенные ниже фрагменты из контекста, чтобы ответить на вопрос. Если ты не знаешь ответ, просто скажи, что не знаешь. Используй максимум три предложения и будь краток."

In [ ]:
# Set up prompt for LLM

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [ ]:
# Create model

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

In [ ]:
# Test model
inp = 'Почему трава зелёная?'

conversation = Conversation()
conversation.add_user_message(inp)
prompt = conversation.get_prompt(tokenizer)
output = generate(model, tokenizer, prompt, generation_config)

print(output)

In [ ]:
# Set question
query1 = "Что находится по адресу 6-я линия В.О., дом 23, литера Б?"
query2 = "Что находится на 6 линии васильевсвого острова 23?"
query3 = "Что находится в доме на московском проспекте 78?"
query4 = "К какому периоду застройки можно отнести дом по адресу Новочеркасский проспект, 41?"
query5 = "Сколько в здании по адресу Новочеркасский проспект, 41 подъездов?"
query6 = "Сколько лифтов в здании по адресу Новочеркасский проспект, 41?"
query7 = "Сколько корпусов у дома с адресом Новочеркасский проспект, 37?"
query8 = "Что такое корпус у здания?"
query9 = "Какие архитектурные стили представлены в Санкт-Петербурге?"
query10 = "Как правильно указать корпус в адресе здания?"
query10 = "Какие стили были распространены в Санкт-Петербурге в 1973 году?"

query = f"Вопрос: {query10}"

# Get context
def get_context_from_db(vectorstore, query):
    docs = vectorstore.similarity_search(query)
    docs = [doc.page_content for doc in docs]
    docs = '\n'.join(docs)
    return docs
    
docs_b = get_context_from_db(buildings_vectorstore, query)
docs_s = get_context_from_db(services_vectorstore, query)
context = f'Контекст: {docs_b}\n{docs_s}'

# Pass context with question to prompt
conversation = Conversation()
conversation.add_user_message(query)
conversation.add_user_message(context)
prompt = conversation.get_prompt(tokenizer)

# Request result
output = generate(model, tokenizer, prompt, generation_config)

print(query)
print(output)